In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
 

In [2]:
datasets.FashionMNIST(root='data')

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train

In [3]:
training_data = datasets.FashionMNIST(root='data',train=True,transform=ToTensor())
test_data = datasets.FashionMNIST(root='data',train=False,transform=ToTensor())
training_data,test_data

(Dataset FashionMNIST
     Number of datapoints: 60000
     Root location: data
     Split: Train
     StandardTransform
 Transform: ToTensor(),
 Dataset FashionMNIST
     Number of datapoints: 10000
     Root location: data
     Split: Test
     StandardTransform
 Transform: ToTensor())

In [4]:
batch_size = 100
train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)
for x,y in test_dataloader:
    print('x:',x.shape)
    print('y:',y.shape)
    break

x: torch.Size([100, 1, 28, 28])
y: torch.Size([100])


## Creating Models

In [5]:
class Network(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = torch.nn.Flatten()
        self.linear_relu = torch.nn.Sequential(
            torch.nn.Linear(28*28,512),
            torch.nn.ReLU(),
            torch.nn.Linear(512,512),
            torch.nn.ReLU(),
            torch.nn.Linear(512,512),
            torch.nn.ReLU(),
            torch.nn.Linear(512,10)
        )
    
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu(x)
        return logits

model = Network().to('cuda')

In [6]:
model.parameters

<bound method Module.parameters of Network(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=10, bias=True)
  )
)>

## Optimizing the Model Parameters


In [7]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)

In [8]:
def train(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch,(x,y) in enumerate(dataloader):
        x,y = x.to('cuda'),y.to('cuda')
        
        pred = model(x)
        loss = loss_fn(pred,y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch %100 == 0:
            loss,current = loss.item(),batch*len(x)
            print(f'loss:{loss:>7f}[{current:>5d}/{size:>5d}]')

In [9]:
def test(dataloader,model,loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for x,y in dataloader:
            x,y = x.to('cuda'),y.to('cuda')
            pred = model(x)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /=num_batches
    correct /= size
    return test_loss, correct

In [10]:
one = 1
two = 0
t = 0
while (one-two)**2 > 0.1e-4:
    t+=1
    print(f'times{t}\n----------------------------')
    train(train_dataloader,model,loss_fn,optimizer)
    test_loss,correct = test(test_dataloader,model,loss_fn)
    if t%2 == 0:
        one = correct
    else:
        two = correct
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
print('done!')

times2
----------------------------
loss:2.301323[    0/60000]
loss:1.081428[10000/60000]
loss:0.848315[20000/60000]
loss:0.558563[30000/60000]
loss:0.562171[40000/60000]
loss:0.601265[50000/60000]
Test Error: 
 Accuracy: 80.6%, Avg loss: 0.546259 

times3
----------------------------
loss:0.387441[    0/60000]
loss:0.362442[10000/60000]
loss:0.496243[20000/60000]
loss:0.426688[30000/60000]
loss:0.474234[40000/60000]
loss:0.498222[50000/60000]
Test Error: 
 Accuracy: 83.9%, Avg loss: 0.450166 

times4
----------------------------
loss:0.293702[    0/60000]
loss:0.325021[10000/60000]
loss:0.435890[20000/60000]
loss:0.355800[30000/60000]
loss:0.469572[40000/60000]
loss:0.476691[50000/60000]
Test Error: 
 Accuracy: 85.2%, Avg loss: 0.413444 

times5
----------------------------
loss:0.248911[    0/60000]
loss:0.304199[10000/60000]
loss:0.399282[20000/60000]
loss:0.323763[30000/60000]
loss:0.469575[40000/60000]
loss:0.443268[50000/60000]
Test Error: 
 Accuracy: 85.8%, Avg loss: 0.393443 



## Saving Models


In [11]:
torch.save(model.state_dict(),'test_model.pth')
print('saved')

saved


## Loading Models


In [12]:
model = Network()
model.load_state_dict(torch.load('test_model.pth'))

<All keys matched successfully>